In [1]:
%run mathutil.ipynb

In [2]:
class Dataset(object):
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
    
    def __str__(self):
        return '{}({}, {}+{}+{})'.format(self.name, self.mode, \
                   len(self.tr_xs), len(self.te_xs), len(self.va_xs))

    #train_count() 메서드는 @property라는 데코레이터를 갖고 있으며
    #이에 따라 실제로는 함수 메서드이면서도 함수가 아닌 속성으로 취급되어 'a.train_count()'형식 대신 인수 구조 없이 'a.train_count'형식으로
    #접근할 수 있게 된다. 물론 이런 메서드는 self 외의 매개변수를 가져서는 안된다.
    @property
    def train_count(self):
        return len(self.tr_xs)

In [3]:
#학습 데이터 공급 메서드 정의
#모델 객체에 미니배치 학습 데이터를 공급하는 역할하는 메서드 정의
def dataset_get_train_data(self, batch_size, nth):
    #미니배치 크기 batch_size와 순번 nth값에 따라 반환 영역을 산정하여
    from_idx = nth * batch_size
    to_idx = (nth + 1) * batch_size

    #미리 준비된 학습 데이터셋 입출력 성분의 일부를 반환한다.
    tr_X = self.tr_xs[self.indices[from_idx:to_idx]]
    tr_Y = self.tr_ys[self.indices[from_idx:to_idx]]

    return tr_X, tr_Y

#학습용 데이터를 뒤섞어주는 메서드 정의
def dataset_shuffle_train_data(self, size):
    #학습용 데이터 부분에 대해서만 새로 뒤섞기를 한다.
    self.indices = np.arange(size)
    np.random.shuffle(self.indices)

Dataset.get_train_data = dataset_get_train_data
Dataset.shuffle_train_data = dataset_shuffle_train_data

In [4]:
#평가 데이터 공급 메서드 정의
def dataset_get_test_data(self):
    return self.te_xs, self.te_ys

Dataset.get_test_data = dataset_get_test_data

In [5]:
#검증 데이터 공급 메서드 정의
def dataset_get_validate_data(self, count):
    self.va_indices = np.arange(len(self.va_xs))
    np.random.shuffle(self.va_indices)

    va_X = self.va_xs[self.va_indices[0:count]]
    va_Y = self.va_ys[self.va_indices[0:count]]

    return va_X, va_Y

Dataset.get_validate_data = dataset_get_validate_data
Dataset.get_visualize_data = dataset_get_validate_data

In [6]:
#데이터 뒤섞기 메서드 정의
#파생 클래스들이 이용할 뒤섞기 메서드 정의
def dataset_shuffle_data(self, xs, ys, tr_ratio=0.8, va_ratio=0.05):
    #tr_ratio,va_ratio값으로 지정한 비율에 따라 학습과 평가 및 검증용 데이터 수를 계산한다.
    data_count = len(xs)

    tr_cnt = int(data_count * tr_ratio / 10) * 10
    va_cnt = int(data_count * va_ratio)
    te_cnt = data_count - (tr_cnt + va_cnt)

    #그런 후에 이들 데이터 수에 따라 세 영역의 구간 시작과 끝 위치를 계산한다.
    tr_from, tr_to = 0, tr_cnt
    va_from, va_to = tr_cnt, tr_cnt + va_cnt
    te_from, te_to = tr_cnt + va_cnt, data_count

    #이어서 난수 함수를 이용해 데이터 뒤섞기용 인덱스를 만들고
    indices = np.arange(data_count)
    np.random.shuffle(indices)

    #준비된 정보에 따라 데이터를 세 영역으로 분할해 저장한다.
    self.tr_xs = xs[indices[tr_from:tr_to]]
    self.tr_ys = ys[indices[tr_from:tr_to]]
    self.va_xs = xs[indices[va_from:va_to]]
    self.va_ys = ys[indices[va_from:va_to]]
    self.te_xs = xs[indices[te_from:te_to]]
    self.te_ys = ys[indices[te_from:te_to]]

    #xs와 ys에서 각각 하나씩 데이터 형태를 조사하여 객체 변수 input_shape와 output_shape에 저장한다.
    self.input_shape = xs[0].shape
    self.output_shape = ys[0].shape
    
    #나중에 정의할 파생 클래스에서 뒤섞인 데이터들의 원래 위치를 알 필요가 있을 때를 대비해
    #세 데이터 영역에 대한 원본의 위치 정보를 반환한다.
    return indices[tr_from:tr_to], indices[va_from:va_to], indices[te_from:te_to]

Dataset.shuffle_data = dataset_shuffle_data

In [7]:
#후처리 순전파 처리 지원 메서드 정의
def dataset_forward_postproc(self, output, y, mode=None):
    #mode 매개변숫값이 None이 아니면 이를 대신 이용한다.
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        #1.10.9절:단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의
        diff = output - y
        square = np.square(diff)
        loss = np.mean(square)
        aux = diff
    elif mode == 'binary':
        #2.9.4절:후처리 과정에 대한 순전파와 역전파 함수의 재정의
        entropy = sigmoid_cross_entropy_with_logits(y, output)
        loss = np.mean(entropy)
        aux = [y, output]
    elif mode == 'select':
        #3.8.4절:후처리 과정에 대한 순전파와 역전파 함수의 재정의
        entropy = softmax_cross_entropy_with_logits(y, output)
        loss = np.mean(entropy)
        aux = [output, y, entropy]
        
    return loss, aux

Dataset.forward_postproc = dataset_forward_postproc

In [8]:
#후처리 역전파 처리 지원 메서드 정의
def dataset_backprop_postproc(self, G_loss, aux, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        #1.10.9절:단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의
        diff = aux
        shape = diff.shape

        g_loss_square = np.ones(shape) / np.prod(shape)
        g_square_diff = 2 * diff
        g_diff_output = 1

        G_square = g_loss_square * G_loss
        G_diff = g_square_diff * G_square
        G_output = g_diff_output * G_diff
    elif mode == 'binary':
        #2.9.4절:후처리 과정에 대한 순전파와 역전파 함수의 재정의
        y, output = aux
        shape = output.shape

        g_loss_entropy = np.ones(shape) / np.prod(shape)
        g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entropy_output * G_entropy
    elif mode == 'select':
        #3.8.4절:후처리 과정에 대한 순전파와 역전파 함수의 재정의
        output, y, entropy = aux

        g_loss_entropy = 1.0 / np.prod(entropy.shape)
        g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)

        G_entropy = g_loss_entropy * G_loss
        G_output = g_entropy_output * G_entropy
    
    return G_output

Dataset.backprop_postproc = dataset_backprop_postproc

In [9]:
#정확도 계산 메서드 정의
def dataset_eval_accuracy(self, x, y, output, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        mse = np.mean(np.square(output - y))
        accuracy = 1 - np.sqrt(mse) / np.mean(y)
    elif mode == 'binary':
        estimate = np.greater(output, 0)
        answer = np.equal(y, 1.0)
        correct = np.equal(estimate, answer)
        accuracy = np.mean(correct)
    elif mode == 'select':
        estimate = np.argmax(output, axis=1)
        answer = np.argmax(y, axis=1)
        correct = np.equal(estimate, answer)
        accuracy = np.mean(correct)
        
    return accuracy

Dataset.eval_accuracy = dataset_eval_accuracy

In [10]:
#추정 결과 변환 메서드 정의
def dataset_get_estimate(self, output, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        #추가 처리 없이 output을 그대로 추정 결과로 삼는다.
        estimate = output
    elif mode == 'binary':
        #로짓값 벡터를 시그모이드 함수를 이용하여 원소 단위의 확률 벡터로 변환
        estimate = sigmoid(output)
    elif mode == 'select':
        #소프트맥스 함수를 이용하여 하나의 확률 분포 벡터로 변환
        estimate = softmax(output)
        
    return estimate

Dataset.get_estimate = dataset_get_estimate

In [11]:
#로그 출력 메서드 정의
def dataset_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    print('    Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)'. \
          format(epoch, np.mean(costs), np.mean(accs), acc, time1, time2))

def dataset_test_prt_result(self, name, acc, time):
    print('Model {} test report: accuracy = {:5.3f}, ({} secs)\n'. \
          format(name, acc, time))

Dataset.train_prt_result = dataset_train_prt_result
Dataset.test_prt_result = dataset_test_prt_result